# Lab 4.5.2: Agent Playground - Solutions

Complete solutions for the Agent Playground exercises.

---

## Solution 1: Streaming Responses in Streamlit

Show tokens as they're generated using `st.write_stream()`.

In [ ]:
streaming_chat = '''
"""Streaming Chat in Streamlit"""

import streamlit as st
import ollama

st.set_page_config(page_title="Streaming Chat", page_icon="⚡")
st.title("⚡ Streaming Chat")

# Initialize messages
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display history
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.write(msg["content"])

# Chat input
if prompt := st.chat_input("Type a message..."):
    # Add user message
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.write(prompt)
    
    # Stream response
    with st.chat_message("assistant"):
        def generate():
            for chunk in ollama.chat(
                model="llama3.2:3b",
                messages=st.session_state.messages,
                stream=True
            ):
                yield chunk["message"]["content"]
        
        # Use st.write_stream for token-by-token display
        response = st.write_stream(generate())
    
    # Save response
    st.session_state.messages.append({"role": "assistant", "content": response})
'''

print("Streaming chat solution:")
print(streaming_chat)

## Solution 2: Tool Latency Tracking

Measure and display tool execution times.

In [ ]:
import time
from typing import Dict, Any, Callable
from functools import wraps


class LatencyTracker:
    """
    Track latency of tool executions.
    """
    
    def __init__(self):
        self.latencies: Dict[str, list] = {}
    
    def track(self, tool_name: str) -> Callable:
        """Decorator to track execution time."""
        def decorator(func: Callable) -> Callable:
            @wraps(func)
            def wrapper(*args, **kwargs):
                start = time.time()
                result = func(*args, **kwargs)
                elapsed = time.time() - start
                
                if tool_name not in self.latencies:
                    self.latencies[tool_name] = []
                self.latencies[tool_name].append(elapsed)
                
                return result
            return wrapper
        return decorator
    
    def get_stats(self, tool_name: str = None) -> Dict:
        """Get latency statistics."""
        if tool_name:
            times = self.latencies.get(tool_name, [])
            if not times:
                return {"error": "No data"}
            return {
                "tool": tool_name,
                "count": len(times),
                "avg_ms": sum(times) / len(times) * 1000,
                "min_ms": min(times) * 1000,
                "max_ms": max(times) * 1000,
            }
        
        # All tools
        return {
            name: self.get_stats(name)
            for name in self.latencies
        }


# Example usage
tracker = LatencyTracker()

@tracker.track("calculator")
def calculate(expression: str) -> str:
    time.sleep(0.1)  # Simulate work
    return str(eval(expression))

@tracker.track("web_search")
def web_search(query: str) -> str:
    time.sleep(0.5)  # Simulate API call
    return f"Results for: {query}"

# Test
calculate("2 + 2")
calculate("10 * 5")
web_search("python")

print("Latency stats:")
print(tracker.get_stats())

In [ ]:
# Streamlit visualization of latencies
latency_page = '''
"""Tool Latency Analytics Page"""

import streamlit as st
import pandas as pd

st.set_page_config(page_title="Tool Latency", page_icon="⏱️")
st.title("⏱️ Tool Latency Analytics")

# Get latency data from session state
if "tool_latencies" not in st.session_state:
    st.session_state.tool_latencies = {}

latencies = st.session_state.tool_latencies

if latencies:
    # Summary metrics
    st.subheader("Summary")
    
    cols = st.columns(len(latencies))
    for i, (tool, times) in enumerate(latencies.items()):
        avg_ms = sum(times) / len(times) * 1000
        cols[i].metric(
            f"{tool}",
            f"{avg_ms:.1f}ms",
            f"{len(times)} calls"
        )
    
    st.subheader("Latency Distribution")
    
    # Create DataFrame for chart
    data = []
    for tool, times in latencies.items():
        for t in times:
            data.append({"Tool": tool, "Latency (ms)": t * 1000})
    
    df = pd.DataFrame(data)
    
    # Box plot style using bar chart
    st.bar_chart(
        df.groupby("Tool")["Latency (ms)"].mean()
    )
    
    # Detailed table
    st.subheader("Detailed Statistics")
    
    stats_data = []
    for tool, times in latencies.items():
        stats_data.append({
            "Tool": tool,
            "Calls": len(times),
            "Avg (ms)": f"{sum(times)/len(times)*1000:.1f}",
            "Min (ms)": f"{min(times)*1000:.1f}",
            "Max (ms)": f"{max(times)*1000:.1f}",
        })
    
    st.dataframe(pd.DataFrame(stats_data), hide_index=True)
    
else:
    st.info("No tool calls recorded yet. Chat with the agent to generate data!")

# Clear button
if st.button("Clear Latency Data"):
    st.session_state.tool_latencies = {}
    st.rerun()
'''

print("Latency analytics page:")
print(latency_page)

## Solution 3: Conversation Branching

Allow users to go back and try different responses.

In [ ]:
branching_chat = '''
"""Conversation Branching in Streamlit"""

import streamlit as st
from dataclasses import dataclass, field
from typing import List, Optional, Dict
import uuid


@dataclass
class Message:
    """A message in the conversation tree."""
    id: str
    role: str
    content: str
    parent_id: Optional[str] = None
    children: List[str] = field(default_factory=list)


class ConversationTree:
    """A tree structure for branching conversations."""
    
    def __init__(self):
        self.messages: Dict[str, Message] = {}
        self.root_id: Optional[str] = None
        self.current_path: List[str] = []  # Current branch
    
    def add_message(self, role: str, content: str, parent_id: str = None) -> str:
        """Add a message to the tree."""
        msg_id = str(uuid.uuid4())[:8]
        msg = Message(
            id=msg_id,
            role=role,
            content=content,
            parent_id=parent_id or (self.current_path[-1] if self.current_path else None)
        )
        
        self.messages[msg_id] = msg
        
        # Set root or add as child
        if msg.parent_id and msg.parent_id in self.messages:
            self.messages[msg.parent_id].children.append(msg_id)
        elif not self.root_id:
            self.root_id = msg_id
        
        self.current_path.append(msg_id)
        return msg_id
    
    def get_current_history(self) -> List[Message]:
        """Get messages in current branch."""
        return [self.messages[mid] for mid in self.current_path]
    
    def branch_from(self, message_id: str):
        """Start a new branch from a specific message."""
        # Find path to this message
        new_path = []
        current = message_id
        
        while current:
            new_path.insert(0, current)
            current = self.messages[current].parent_id
        
        self.current_path = new_path
    
    def get_branch_points(self) -> List[Dict]:
        """Get all messages with multiple children (branch points)."""
        return [
            {
                "id": msg.id,
                "content": msg.content[:50] + "...",
                "branches": len(msg.children)
            }
            for msg in self.messages.values()
            if len(msg.children) > 1
        ]


# Streamlit App
st.set_page_config(page_title="Branching Chat", page_icon="🌳")
st.title("🌳 Branching Chat")

# Initialize tree
if "tree" not in st.session_state:
    st.session_state.tree = ConversationTree()

tree = st.session_state.tree

# Sidebar: Branch navigation
with st.sidebar:
    st.header("🌿 Branches")
    
    branch_points = tree.get_branch_points()
    if branch_points:
        st.write("Messages with branches:")
        for bp in branch_points:
            if st.button(f"↩️ {bp[\'content\']}", key=bp["id"]):
                tree.branch_from(bp["id"])
                st.rerun()
    else:
        st.info("No branches yet. Try regenerating a response!")
    
    st.markdown("---")
    st.write(f"Messages: {len(tree.messages)}")
    st.write(f"Current path: {len(tree.current_path)}")

# Display current branch
for msg in tree.get_current_history():
    with st.chat_message(msg.role):
        st.write(msg.content)
        
        # Show branch indicator
        if len(msg.children) > 1:
            st.caption(f"🌿 {len(msg.children)} branches")
        
        # Regenerate button for assistant messages
        if msg.role == "assistant" and msg.id == tree.current_path[-1]:
            if st.button("🔄 Try different response", key=f"regen_{msg.id}"):
                # Go back to parent and regenerate
                tree.branch_from(msg.parent_id)
                st.session_state.regenerate = True
                st.rerun()

# Chat input
if prompt := st.chat_input("Type a message..."):
    tree.add_message("user", prompt)
    
    # Generate response (mock)
    import random
    responses = [
        f"Response A: {prompt}",
        f"Response B: {prompt}",
        f"Response C: {prompt}"
    ]
    response = random.choice(responses)
    tree.add_message("assistant", response)
    
    st.rerun()
'''

print("Branching chat solution:")
print(branching_chat[:2000] + "...")

## Solution 4: Dynamic Tool Creation

Allow users to define new tools through the UI.

In [ ]:
dynamic_tools = '''
"""Dynamic Tool Creation Page"""

import streamlit as st
import json

st.set_page_config(page_title="Tool Builder", page_icon="🔨")
st.title("🔨 Dynamic Tool Builder")

st.markdown("""
Create custom tools for the agent. Define the tool\'s name, description,
and a Python function to execute.
""")

# Initialize custom tools
if "custom_tools" not in st.session_state:
    st.session_state.custom_tools = {}

# Tool creation form
with st.expander("➕ Create New Tool", expanded=True):
    with st.form("new_tool"):
        tool_name = st.text_input(
            "Tool Name",
            placeholder="my_custom_tool",
            help="Use lowercase with underscores"
        )
        
        tool_description = st.text_area(
            "Description",
            placeholder="What does this tool do?",
            help="Clear description helps the agent decide when to use it"
        )
        
        tool_params = st.text_area(
            "Parameters (JSON)",
            value=\'\'\'{"param1": "string", "param2": "number"}\'\'\',
            help="Define input parameters as JSON"
        )
        
        tool_code = st.text_area(
            "Python Code",
            value="""def execute(params):
    # Access params like: params["param1"]
    result = f"Processed: {params}"
    return result""",
            height=200,
            help="Define the execute function"
        )
        
        submitted = st.form_submit_button("Create Tool")
        
        if submitted and tool_name:
            try:
                # Validate JSON
                params = json.loads(tool_params)
                
                # Store tool definition
                st.session_state.custom_tools[tool_name] = {
                    "name": tool_name,
                    "description": tool_description,
                    "parameters": params,
                    "code": tool_code
                }
                
                st.success(f"Tool \'{tool_name}\' created!")
            except json.JSONDecodeError:
                st.error("Invalid JSON in parameters")

# List existing custom tools
st.markdown("---")
st.subheader("📋 Custom Tools")

if st.session_state.custom_tools:
    for name, tool in st.session_state.custom_tools.items():
        with st.expander(f"🔧 {name}"):
            st.markdown(f"**Description:** {tool[\'description\']}")
            st.markdown("**Parameters:**")
            st.json(tool["parameters"])
            st.markdown("**Code:**")
            st.code(tool["code"], language="python")
            
            col1, col2 = st.columns(2)
            with col1:
                if st.button("Test", key=f"test_{name}"):
                    # Execute the tool with sample params
                    try:
                        exec_globals = {}
                        exec(tool["code"], exec_globals)
                        result = exec_globals["execute"]({
                            k: f"test_{k}" for k in tool["parameters"]
                        })
                        st.success(f"Result: {result}")
                    except Exception as e:
                        st.error(f"Error: {e}")
            
            with col2:
                if st.button("Delete", key=f"del_{name}"):
                    del st.session_state.custom_tools[name]
                    st.rerun()
else:
    st.info("No custom tools yet. Create one above!")

# Export tools
st.markdown("---")
if st.session_state.custom_tools:
    export_data = json.dumps(st.session_state.custom_tools, indent=2)
    st.download_button(
        "📥 Export Tools",
        data=export_data,
        file_name="custom_tools.json",
        mime="application/json"
    )
'''

print("Dynamic tool creation page:")
print(dynamic_tools[:2000] + "...")

## Key Takeaways

1. **Streaming** dramatically improves perceived responsiveness
2. **Latency tracking** helps identify performance bottlenecks
3. **Conversation branching** enables exploration of alternatives
4. **Dynamic tools** make the agent extensible by users

These patterns make your agent playground more powerful and user-friendly.